<a href="https://colab.research.google.com/github/pabloac31/TFG/blob/master/ImageNet_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/pabloac31/TFG.git
%cd TFG

Cloning into 'TFG'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 163 (delta 75), reused 43 (delta 19), pack-reused 41
Receiving objects: 100% (163/163), 79.17 MiB | 10.54 MiB/s, done.
Resolving deltas: 100% (81/81), done.
/content/TFG


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from utils import *
from adversarial_attacks import *
import torchvision.models as models

In [4]:
iv3 = models.inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth


In [5]:
# Define what device we are using
use_cuda=True
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True


In [0]:
mean_ImageNet = [0.485, 0.456, 0.406]
std_ImageNet = [0.229, 0.224, 0.225]

def testloader_ImageNet(path, batch_size, shuffle=True):
  transform = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_ImageNet, std=std_ImageNet),
  ])

  test_loader = torch.utils.data.DataLoader(
      datasets.ImageNet(root=path, split='val', transform=transform, download=True),
      batch_size=batch_size, shuffle=shuffle
  )

  return test_loader

In [10]:
test_loader = testloader_ImageNet('./data/', 1)

RuntimeError: ignored